In [1]:
import numpy as np
import pandas as pd

# method to calculate similarity between two arrays
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# Used id columns to concatinate with match matrix
user_id = pd.read_csv('../data/id.csv')

# File containing sex and orientation feature of each user
pref = pd.read_csv('../data/preference.csv')

# File with preprocessed and vectorised features of each user
df = pd.read_csv('../data/refined_data.csv')

In [3]:
pref.head()

,sex,orientation
0,f,gay
1,m,gay
2,f,bisexual
3,m,bisexual
4,f,bisexual


In [4]:
df.head()

,age,height,education_level,status,drinks,drugs,job,location,pets,smokes,...,zombie,zombieland,zone,zoo,zookeeper,zooming,zu,zumba,zurich,zynga
0,27,66.0,4.0,3,4,0,10,40,14,0,...,0,0,0,0,0,0,0,0,0,0
1,26,68.0,3.0,3,4,0,12,47,11,0,...,0,0,0,0,0,0,0,0,0,0
2,20,69.0,2.0,2,4,2,12,40,9,1,...,0,0,0,0,0,0,0,0,0,0
3,27,68.0,3.0,3,4,2,3,14,14,0,...,0,0,0,0,0,0,0,0,0,0
4,22,68.0,2.0,3,2,2,12,40,14,4,...,0,0,0,0,0,0,0,0,0,0


In [5]:
# Converting DataFramme into array

X = df.values
X

array([[27., 66.,  4., ...,  0.,  0.,  0.],
       [26., 68.,  3., ...,  0.,  0.,  0.],
       [20., 69.,  2., ...,  0.,  0.,  0.],
       ...,
       [28., 71.,  3., ...,  0.,  0.,  0.],
       [32., 68.,  3., ...,  0.,  0.,  0.],
       [41., 67.,  2., ...,  0.,  0.,  0.]])

In [6]:
# Determine unique orientations

pref.orientation.unique()

array(['gay', 'bisexual', 'straight'], dtype=object)

In [ ]:
# Basic idea for calculating match is as per the user's sexual orientation:
# 1) if a user is straight, then calculate the match percentage if the other user is of opposite gender and has straigh or bisexual
# sexual orientation else put 0.
# 2) if a user is gay, then calculate the match percentage if the other user is gay or bisexual and of same gender else put 0.
# 3) if a user is bisexual then calculate the match percentage if the other user is bisexual/ straight and of opposite gender/ gay
# and of same gender else put 0.
# 4) if both users are same thn put 0.



match = [] # Match matrix
for i in range(len(df)):
    sex_i = pref.sex[i] # sex of first user
    pref_i = pref.orientation[i] # orientation of first user
    l = []
    for j in range(len(df)):
        sex_j = pref.sex[j] # sex of second user
        pref_j = pref.orientation[j] # orientation of second user
        if i != j: 
#           For gay
            if pref_i=='gay' :
                if sex_i == sex_j and (pref_j in ['gay','bisexual']):
                    l.append(cosine_similarity([X[i]],[X[j]])[0][0]*100)
                else:
                    l.append(0)
            
#           For straight
            elif pref_i=='straight' :
                if sex_i != sex_j and (pref_j in ['straight','bisexual']):
                    l.append(cosine_similarity([X[i]],[X[j]])[0][0]*100)
                else:
                    l.append(0)

#           For bisexuals
            elif pref_i=='bisexual' :
                if pref_j == 'straight' and sex_i != sex_j:
                    l.append(cosine_similarity([X[i]],[X[j]])[0][0]*100)
                elif pref_j == 'gay' and sex_i == sex_j:
                    l.append(cosine_similarity([X[i]],[X[j]])[0][0]*100)
                elif pref_j == 'bisexual':
                    l.append(cosine_similarity([X[i]],[X[j]])[0][0]*100)
                else:
                    l.append(0)
            
        else:
            l.append(0)
    match.append(l)
    print(i,end=' ') # just to print the index of first user to determine progress

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 

In [8]:
# Concatenate the user is with match matrix in sinngle dataframe

sub = pd.concat((user_id,pd.DataFrame(match,columns=user_id.user_id)),axis=1)
sub.head()

,user_id,fffe3100,fffe3200,fffe3300,fffe3400,fffe3500,fffe3600,fffe3700,fffe3800,fffe3900,...,fffe3100390039003200,fffe3100390039003300,fffe3100390039003400,fffe3100390039003500,fffe3100390039003600,fffe3100390039003700,fffe3100390039003800,fffe3100390039003900,fffe3200300030003000,fffe3200300030003100
0,fffe3100,0.000000,0.000000,97.272958,0.000000,96.372671,0.000000,96.388216,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,95.808761,0.000000,0.000000,0.000000
1,fffe3200,0.000000,0.000000,0.000000,88.493015,0.000000,92.500712,0.000000,0.000000,94.058124,...,94.655325,0.000000,95.034795,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,fffe3300,97.272958,0.000000,0.000000,92.020119,98.655817,95.360617,94.161594,93.705576,0.000000,...,0.000000,89.278953,0.000000,0.000000,95.217656,96.929825,97.470613,97.346639,97.125258,0.000000
3,fffe3400,0.000000,88.493015,92.020119,0.000000,91.138404,96.664568,0.000000,0.000000,87.213692,...,87.638875,0.000000,87.119558,84.298073,0.000000,0.000000,0.000000,0.000000,0.000000,92.011155
4,fffe3500,96.372671,0.000000,98.655817,91.138404,0.000000,93.575868,93.907717,94.753635,0.000000,...,0.000000,88.875254,0.000000,0.000000,95.867940,98.167513,97.510559,96.885696,97.018955,0.000000


In [9]:
# Savnig match matrix into csv file format

sub.to_csv('../data/sub_lbl.csv')

In [31]:
path = '../models/le_job.pkl'

In [34]:
import dill
a = dill.load(open(path,'rb'))

In [35]:
a.classes_

array(['artistic / musical / writer', 'banking / financial / real estate',
       'clerical / administrative', 'computer / hardware / software',
       'construction / craftsmanship', 'education / academia',
       'entertainment / media', 'executive / management',
       'hospitality / travel', 'law / legal services',
       'medicine / health', 'military', 'other', 'political / government',
       'rather not say', 'retired', 'sales / marketing / biz dev',
       'science / tech / engineering', 'student', 'transportation',
       'unemployed'], dtype=object)